TfIdf Training

In [ ]:
import os

os.chdir(os.getcwd() + "\\..")

from engine.notebook_utils import *
from engine.utils import *
from sklearn.metrics.pairwise import cosine_similarity


documents_metadata = fetch_training_dataset1_metadata(limit=None)


print(f"Corpus size = {len(documents_metadata)}")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = []
documents_ids = []
for d in documents_metadata:
    content = load_document_content(d.key)
    documents.append(content)
    documents_ids.append(d.id)


tfidf_vectorizer = TfidfVectorizer(analyzer='word', preprocessor=preprocess, tokenizer=tokenize_content)

tfidf = tfidf_vectorizer.fit_transform(documents)

In [ ]:

import os

save_model(tfidf_vectorizer, os.path.join("tfidf", "vectorizer.pk"))

save_model(tfidf, os.path.join("tfidf", "matrix.pk"))

save_model(documents_ids, os.path.join("tfidf", "documents_ids.pk"))

Evaluation

In [ ]:
os.chdir(os.getcwd() + "\\..")
from evaluation import *
from engine.tfidf_engine import TfIdfEngine
from engine.notebook_utils import get_test_queries_path, get_test_queries_matches_path

test_queries_path = get_test_queries_path()
test_queries_matches_path = get_test_queries_matches_path()

engine = TfIdfEngine(threshold=0.12)

def listener(query_id, precision, recall, avg_precision, precision_at_10):
    print(f"query id: {query_id}, recall: {recall}, precision: {precision}, precision@10: {precision_at_10}, avg_precision: {avg_precision}")

evaluation = evaluate(engine, test_queries_path, test_queries_matches_path, listener)


In [ ]:
import pandas as pd

df = pd.DataFrame(evaluation['queries'])

map_mrr_df = pd.DataFrame([
    {
        'Average Recall': evaluation['average_recall'],
        'Average Precision': evaluation['average_precision'],
        'MAP': evaluation['mean_average_precision'],
        'MRR': evaluation['mean_reciprocal_rank'],
    }
])

print(df)
print()
print(map_mrr_df)